## Word Count Tutorial

The Konfuzio SDK allows you to post annotations in your documents in your Konfuzio project. Our Konfuzio server allows you to download these annotations in a structured way which enables you to analyse the dataset. This can be used for multiple applications, e.g. to analyze on which subjects companies focus their attentions and investments.  
This tutorial guides you through how to count certain, pre-specified expressions. This will be done by showing a specific use case by analyzing how much sustainability plays a role in the annual reports of german companies over a certain time span, particularly in relation to the appearance of the Covid-19 pandemic. This information can then be used to further proceed with your dataset analysis.

### 1. Install and initialize the Konfuzio SDK package
Start by installing the Konfuzio SDK package.  
To use the package, please also register or sign in on app.konfuzio.com.
Find more information in the readme of the [repository](https://github.com/konfuzio-ai/document-ai-python-sdk).

In [ ]:
!pip install konfuzio-sdk

In [ ]:
!konfuzio_sdk init

Note:  
If you want to switch to a different project, you have to restart the variables in colab (run cell bellow).

In [ ]:
!kill -9 -1

You also need to import the required packages shown below.

In [ ]:
import re
import json
import matplotlib.pyplot as plt
import pandas as pd
from konfuzio_sdk import SUPPORTED_FILE_TYPES
from konfuzio_sdk.api import konfuzio_session
from konfuzio_sdk.data import Project, Annotation, Document, Label, Section
from konfuzio_sdk.utils import get_file_type

### 2. Initiate the project 
Initiating the project might take some time if you use huge file volumes.  
The files will be stored locally in the folder you specified before which is "data" by default.

In [ ]:
my_project = Project()

Upload your documents using the following code.

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# filter the documents by the supported file types
import glob
import os
files_to_check = [f for f in glob.glob('*') if os.path.isfile(f) and not f.endswith('.py')]
files_in_dir = []

for file_path in files_to_check:
    try:
        file_type = get_file_type(file_path)
    if file_type in SUPPORTED_FILE_TYPES:
        files_in_dir.append(file_path)
    except NotImplemented:
        continue

In [ ]:
for doc_file in files_in_dir:
    Document(file_path=doc_file, project=my_project).save()

Note:  
If you prefer, use the example documents in the example folder with sample documents provided [here](https://drive.google.com/drive/folders/1iWPBdxHqR8j8CWUdbVHO7OzIxZsOXe-l).

The documents are annual reports from companies which are part of the prime standard (full list of companies in prime standard can be found [here](https://www.deutsche-boerse-cash-market.com/dbcm-de/instrumente-statistiken/statistiken/gelistete-unternehmen)).   

Alternatively, you can do this step directly on the Konfuzio server by uploading the documents there.  
Follow the instructions in the readme of the [repository](https://github.com/konfuzio-ai/document-ai-python-sdk).  

Update your project to have the information of the documents after being processed.

In [ ]:
my_project.update()

Get the documents from the project.

In [ ]:
documents = my_project.get_documents_from_project()

Check how many documents there are.

In [ ]:
print(len(documents))

### 3. Specify the expressions you want to count
Our aim is to count expressions related to sustainability (here: _climate_expressions_) and expressions related to pandemics (here: _pandemic_expressions_).  
Hence, we specify certain words and expressions related to these subjects.   These expressions will then be searched in the documents of the project to find exact matches. However, there is flexibility for lower- or uppercases. 

In [ ]:
# List of climate expressions to match
climate_expressions = [re.compile("climate change|climate crisis|climate emergency|climate breakdown", re.I),
                       re.compile("global warming|global heating", re.I),
                       re.compile("Greenhouse Gas", re.I),
                       re.compile("carbon emission", re.I),
                       re.compile("renewable energy|renewable sources", re.I)]

In [ ]:
# List of pandemic expressions to match
pandemic_expressions = [re.compile("Corona", re.I),
                     re.compile("Covid", re.I),
                     re.compile("pandemic", re.I),
                     re.compile("lockdown", re.I)]

### 4. Define the labels in the project

As we have expressions related to different subjects, we want to create a label for each one.  

To do this, start by setting up a template on app.konfuzio.com (_Templates -> + Add Template_). 
It is important and necessary that each new annotation gets assigned to a new section. Therefore, tick the box "Has multiple sections" to achieve this. 

Update your project to get the new template.

In [ ]:
my_project.update()

Get the ID of the template.

In [ ]:
template = [template for template in my_project.templates if not template.is_default][0]

Now, you can add labels to your respective template.  
These labels serve as categories or containers for your annotations. In our example, we cluster our annotations into expressions either related to pandemic or to sustainability. Therefore, we create two labels "Pandemic" and "Climate" to allocate the annotations accordingly. Down below, we define the label names of the project we want to use and match them with the existing ones in our project for the further proceeding.


In [ ]:
# labels for which we want to cluster the expressions
labels_names = ['Pandemic', 'Climate']

In [ ]:
for label in labels_names:
    Label(project=my_project, text=label, templates=[template]).save()

To confirm that the labels exist in the project:

In [ ]:
my_project.update()

print(f"Current labels in your project:\n{my_project.labels}")

Collect the defined labels and the correspondent template.

In [ ]:
# labels defined in the Konfuzio project that match the ones we want
labels = [label for label in my_project.labels if label.name in labels_names]

# correspondent templates
templates = [my_label.templates[0] for my_label in labels]

### 5. Find matches for the expressions in the documents
Now we find expressions in the documents which exactly match the expressions we defined above.  
If there is a match, it will be saved as an annotation in a separate section in the documents and can be viewed on app.konfuzio.com in the smart view (https://app.konfuzio.com/admin/server/document/) of the respective document.  
If there are separate expressions next to each other in one document, they are counted as separate ones although they could be viewed as one single expression.

In [ ]:
# find matches in each document and for each label for the expressions defined above, if not already done before and save the annotations in the corresponding documents:
expressions = {'Climate': climate_expressions, 'Pandemic': pandemic_expressions}

for document in documents:
    for label, template in zip(labels, templates):
    
        for exp in expressions[label.name]:
            matches = [(m.start(0), m.end(0)) for m in re.finditer(exp, document.text)]
        
            for match in matches:
                annotation_obj = Annotation(
                    document=document,
                    section=None,
                    document_id=document.id,
                    start_offset=match[0],
                    end_offset=match[1],
                    label=label,
                    template_id=template.id,
                    accuracy=1.0,
                    is_correct=True,
                    revised=True,
                )
                
                _ = annotation_obj.save()

After the annotations have been created online, update the documents with the posted annotations. 

In [ ]:
for document in documents:
    document.get_document_details(update=True)

Let's check how many matches were found in the first document:

In [ ]:
print(len(documents[0].annotations()))

### 6. Data analysis

Let's have a quick look into how many matches were found.

The following provides you with the annotations per label for each of our respective documents separately, giving you the opportunity to analyze the annotations for the individual documents. 

In [ ]:
# count of annotations per label and per document:
print("Annotations per document and per label:")
for doc in documents:
    count_dict_doc= {}
    print("\n{}:".format(doc))

    for annotation in doc.annotations():
        label_name = annotation.label.name
        if label_name in labels_names:
            if label_name in count_dict_doc.keys():
                count_dict_doc[label_name] += 1
            else:
                count_dict_doc[label_name] = 1
                
    print(count_dict_doc)

The Annotations are now also posted on app.konfuzio.com. You can view them in the smart view there. 
To do a more complete and extensive analysis and to get a first glance about what is possible with the results we achieved with this tutorial, you can either use the provided code down below or you can export the information from the Konfuzio server by downloading the data as a csv file.

To download the csv file, please follow these steps:
- Either select certain documents by ticking the boxes of the individual documents or filtering them with the filtering options on the right, e.g. by dataset status: training
- Or select all documents of your project by clicking on "select all (...) documents" on the bottom right corner below the documents
- Select the action "get all data as csv file." in the dropdown menu

This csv provides you with further possibilities to work with the data and draw conclusions from it.

In this example we choose to analyze the relation between the occurences of sustainability and pandemic expressions, hence whether companies decreased their attention to sustainability issues with the arise of a global pandemic. In the code provided below, you can take a look at a first analysis drawn from the data we achieved with this analysis of the annual reports of the companies. 

Retrieve different information from the documents of your Konfuzio project and create a dataframe with these information for a better overview:

In [ ]:
docs_info = []

for doc in documents:
    doc_info = {'name': doc.name,
                'ISIN': doc.name.split("_")[0],
                'year': doc.name.split("_")[1],
                'ID': doc.id,
                'URL': doc.file_url,
                'status': doc.dataset_status,
                'nopages': doc.number_of_pages}
    
    docs_info.append(doc_info)

df = pd.DataFrame(data=docs_info)

In the next step, retrieve also the annotations per label. In our case, the count of climate and pandemic expressions and add them with separate columns to the dataframe:

In [ ]:
total_climate = []
total_pandemic = []

for doc in documents:
    annot = doc.annotations()
    count_climate = len([a for a in annot if a.label.name == 'Climate'])
    total_climate.append(count_climate)
    count_pandemic = len([a for a in annot if a.label.name == 'Pandemic'])
    total_pandemic.append(count_pandemic)

df["topics_climate"] = total_climate
df["topics_pandemic"] = total_pandemic
df.head()

In [ ]:
sum_pages = df['no. pages'].sum()
print("Total sum of pages in dataset: {}.".format(sum_pages))
sum_topics_climate = df["topics_climate"].sum()
print("Total sum of topics_climate: {}.".format(sum_topics_climate))
sum_topics_pandemic = df["topics_pandemic"].sum()
print("Total sum of topics_pandemic: {}.".format(sum_topics_pandemic))

Retrieve the total number of pages of all annual reports present in the dataset as well as the total count of climate and pandemic expressions. Furthermore, group the number of climate and pandemic expressions by year to analyze the development of occurrences of each one over the timespan from 2010 to 2020:

In [ ]:
def count_per_year(dataframe, label):
    grouped = dataframe.groupby('year')['topics_' + label].sum()
    grouped_df = {'count_' + label: grouped}
    grouped_df = pd.DataFrame(data=grouped_df)
    grouped_df.plot(kind = 'bar')
    plt.title(f'Number of {label} expressions per year')

In [ ]:
count_per_year(df, 'climate')
count_per_year(df, 'pandemic')

As we can recognize, sustainability still played an important role for companies with the issue even increasing in importance and occurence over the years. As expected, pandemic expressions strongly rose with the beginning and emergence of the Covid-19 pandemic.
For further background information and additional application possibilities, as well as specific analysises of the different sectors during the timespan analyzed, we kindly want to refer to our blogpost related to this tutorial: _link to blogpost will be inserted here when published_.